In [ ]:
import polars as pl
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()
from linknode_graph_patterns import LinkNodeGraphPatterns

def graphToDataFrame(_g_):
    _lu_ = {'fm':[], 'to':[]}
    for _e_ in _g_.edges: _lu_['fm'].append(_e_[0]), _lu_['to'].append(_e_[1])
    return pl.DataFrame(_lu_)
_dfs_, _patterns_, _node_colors_ = [], LinkNodeGraphPatterns(), {}
for _pattern_ in _patterns_.types:
    _g_  = _patterns_.createPattern(_pattern_, prefix=_pattern_+'__')
    for _node_ in _g_.nodes: _node_colors_[_node_] = _pattern_
    _dfs_.append(graphToDataFrame(_g_))
df    = pl.concat(_dfs_)
g     = rt.createNetworkXGraph(df, [('fm','to')])
_pos_ = nx.spring_layout(g) # rtsvg.PolarsForceDirectedLayout(g).results()

In [ ]:
pos_adj, shapes_gen = rt.circlePackTheSeparateComponents(g, _pos_)
pos_tm              = rt.treeMapGraphComponentPlacement(g, _pos_)
_params_ = {'df':df, 'relationships':[('fm','to')], 'w':512, 'h':512, 'node_size':3, 'node_color':_node_colors_}
rt.tile([rt.link(**_params_, pos=_pos_),rt.link(**_params_, pos=pos_adj, bg_shape_lu=shapes_gen), rt.link(**_params_, pos=pos_tm)])